In [1]:
from ETL.Pipeline import Pipeline, FlowConfig
from ETL.modules.DataSource import DataSourceConfig
from ETL.modules.Encoder import EncoderConfig
from ETL.modules.Loader import LoaderConfig
from ETL.modules.Utils import Utils
from datetime import datetime

# Datasource Config

ds_config = DataSourceConfig().mysql_source_config("127.0.0.1", "root", "", "mt-engineering", "Select * from wp_postmeta")

# Encoder Config

cols_source = ['meta_id', 'post_id', 'meta_key', 'meta_value']
cols_dest = ['meta_id', 'post_id', 'meta_key', 'META_VALUE_UPPER']
en_config = EncoderConfig().mysql_encoder_config(cols_source, cols_dest)

# Transformation Function

def upper_meta_value(df):
    df['meta_value'] = df['meta_value'].str.upper()
    return df

# Loader Config

ld_config = LoaderConfig().json_file_loader(f'export/mysql_fetch_example.json')

# Control Config - Create Started date/time

date_string = "2023-11-01 00:00:00"
date_format = "%Y-%m-%d %H:%M:%S"
start_datetime = datetime.strptime(date_string, date_format)

# Pipeline Object Creation

pipeline = Pipeline("ETL Pipeline")

# Create Pipeline Elements

## Flow 1

pipeline.create_datasource('MySQL', ds_config)
pipeline.create_encoder('wp_postmeta_column', en_config)
pipeline.create_ingestion('Ingestion from 127.0.0.1/mt-engineering')
pipeline.create_control('30 Seconds Update',30,3600,start_datetime)
pipeline.create_process('Upper Meta Value', [Utils().load_obj('./upper_meta_value.pkl')])
pipeline.create_loader('Json File Loader on export/mysql_fetch_example.json', ld_config)

# Flow Config

flow_config = FlowConfig()

flow1 = flow_config.create_flow(
    ID_datasource='MySQL',
    ID_encoder='wp_postmeta_column',
    ID_ingestion='Ingestion from 127.0.0.1/mt-engineering',
    ID_control='30 Seconds Update',
    ID_process='Upper Meta Value',
    ID_loader='Json File Loader on export/mysql_fetch_example.json')

flow_config.add_flow(flow1)

# Set Pipeline flow

pipeline.set_flow(flow_config.get_flow_config())

# Pipeline Start 

pipeline.start()

Linking Pipeline flows for ETL Pipeline
Creating Thread...
Wait for first execution at 2023-11-03 17:21:00 for Ingestion from 127.0.0.1/mt-engineering


In [1]:
import pandas as pd
from modules.Utils import Utils

df = pd.DataFrame(Utils().read_json('export/mysql_fetch_example.json'))

df

,meta_id,post_id,meta_key,META_VALUE_UPPER
0,1,2,_wp_page_template,DEFAULT
1,2,3,_wp_page_template,DEFAULT
2,30,18,_wp_attached_file,WOOCOMMERCE-PLACEHOLDER.PNG
3,31,18,_wp_attachment_metadata,"A:6:{S:5:""WIDTH"";I:1200;S:6:""HEIGHT"";I:1200;S:..."
4,38,26,_elementor_edit_mode,BUILDER
...,...,...,...,...
2268,2648,6125,_elementor_css,"A:6:{S:4:""TIME"";I:1697463737;S:5:""FONTS"";A:1:{..."
2269,2649,1583,_elementor_css,"A:6:{S:4:""TIME"";I:1697463792;S:5:""FONTS"";A:1:{..."
2270,2650,1476,_elementor_css,"A:6:{S:4:""TIME"";I:1697463838;S:5:""FONTS"";A:0:{..."
2271,2651,1825,_elementor_css,"A:6:{S:4:""TIME"";I:1697463870;S:5:""FONTS"";A:1:{..."
